In [ ]:
# copy files
!mkdir data
!mkdir data/out 
!mkdir data/test
!mkdir data/train
!mkdir data/weights
!cp ../input/santa-train-images/train.csv data/
!cp -r ../input/santa-train-images/train/* ./data/train/

In [ ]:
# copy random images from train to test dir
import os
import shutil
import random
files = [file for file in os.listdir('./data/train/') if os.path.isfile(os.path.join('./data/train/', file)) if not file.startswith('.')]
random_amount = 200
for x in range(random_amount):
    if len(files) == 0:
        break
    else:
        file = random.choice(files)
        shutil.copy2(os.path.join('./data/train/', file), './data/test')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import tensorflow_hub as hub
import datetime

import os

In [ ]:
data_path = './data/train/'
train_labels = pd.read_csv('./data/train.csv', dtype=str, sep='\t')

In [ ]:
batch_size = 128
img_height = 224
img_width = 224

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.1)
train_ds = train_datagen.flow_from_dataframe(train_labels, directory = data_path, x_col = 'image_name', y_col = 'class_id',target_size=(img_height, img_width), batch_size=batch_size, shuffle=True, subset="training")
val_ds = train_datagen.flow_from_dataframe(train_labels, directory = data_path, x_col = 'image_name', y_col = 'class_id',target_size=(img_height, img_width), batch_size=batch_size, shuffle=True, subset="validation")

In [ ]:
test_dir = './data/test'
predict_ds = tf.keras.utils.image_dataset_from_directory(test_dir,seed=123,label_mode=None,image_size=(img_height, img_width),batch_size=batch_size, shuffle=False)

In [ ]:
batch = train_ds[0]
images = batch[0]
labels = batch[1]

plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].astype("uint8"))
    plt.title(labels[i])
    plt.axis("off")

In [ ]:
batch = val_ds[0]
images = batch[0]
labels = batch[1]

plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].astype("uint8"))
    plt.title(labels[i])
    plt.axis("off")

In [ ]:
plt.figure(figsize=(10, 10))
for images in predict_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
mobilenet_v3_large_100_224 = "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5"
inception_v3 = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"
mobilenet_v3_large_100_224_c = "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/classification/5"

feature_extractor_model = mobilenet_v3_large_100_224
feature_extractor_model_c = mobilenet_v3_large_100_224_c

In [ ]:
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),
        layers.RandomTranslation(width_factor=0.2, height_factor=0.2)
    ]
)

normalization_layer = tf.keras.layers.Rescaling(1./255)

In [ ]:
num_classes = 3

model = tf.keras.Sequential([
    data_augmentation,
    normalization_layer,
    feature_extractor_layer,
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dense(num_classes)
])
model.build((batch_size,224,224,3))
model.summary()

In [ ]:
base_learning_rate = 0.0001
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
tf.keras.utils.plot_model(model, to_file='plot_img.png', show_shapes=True)

In [ ]:
num_epochs = 60

history = model.fit(train_ds,
                    validation_data = val_ds,
                    epochs=num_epochs,
                    callbacks=tensorboard_callback)

In [ ]:
model.save('./data/weights/tf_model')

In [ ]:
feature_extractor_layer.trainable = True

In [ ]:
model.compile(
  optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

In [ ]:
num_epochs = 30

history = model.fit(train_ds,
                    validation_data = val_ds,
                    epochs=num_epochs,
                    callbacks=tensorboard_callback)

In [ ]:
model.save('./data/weights/tf_model_f')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(num_epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model = keras.models.load_model('./data/weights/tf_model')
preds_all = None
for item in predict_ds:
    preds = model.predict(item)
    preds = np.argmax(preds,axis=1)
    if preds_all is not None:
        preds_all = np.concatenate([preds_all, preds])
    else:
        preds_all = preds.copy()  

In [ ]:
labels = (train_ds.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in preds_all]
submit = pd.DataFrame({'image_name': [path.split('/')[-1] for path in predict_ds.file_paths], 'class_id': predictions})
submit.to_csv('./data/out/submission.csv', sep='\t', index=False)

In [ ]:
# !cp -r ./data/weights/tf_model ./
import shutil
shutil.make_archive('tf_model', 'zip', './data/weights/tf_model')

In [ ]:
# import os -> os.chdir(r'/kaggle/working')
# or
# %cd /kaggle/working
# Now save your required files in this Directory .
# Then run the following cell
# from IPython.display import FileLink -> FileLink(r'*name of file*')

In [ ]:
!rm -r ./data/train
!rm -r ./data/test
!rm -r ./logs

In [ ]:
fe_model = tf.keras.applications.MobileNetV3Large(include_top=False, weights='imagenet', input_shape= (224, 224, 3))
tf.keras.utils.plot_model(fe_model, to_file='plot_img.png', show_shapes=True)